
## Notebook for kjøring av NorMistral-7b-warm-instruct

Authors: Bjarki Thor Norddahl, Victoria Kolsing, Magnus Kristian Wiik

Åpne opp https://colab.research.google.com/ og legg til denne notebooken der.
Husk å bytte Runtime type til T4 GPU

In [1]:
!pip install bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 11.3 MB/s eta 0:00:00


In [2]:
import torch
from google.colab import output
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown, Latex


model_name = 'norallm/normistral-7b-warm-instruct'
tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    low_cpu_mem_usage=True,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16
)
output.clear()

# Chat without history

In [3]:
# ChatMl format
prompt = """<|im_start|> user {input}<|im_end|>
<|im_start|> assistant
"""

# A function that will take care of generating the output
@torch.no_grad()
def generate(input_dict):
    text = prompt.format(**input_dict)
    input_ids = tokenizer(text, return_tensors='pt').input_ids.cuda()
    prediction = model.generate(
        input_ids,
        max_new_tokens=1024, #adjust up or down to 4096
        temperature=0.2, # adjust up or down for creativity
        top_p=0.95, # adjust up or down for creativity
        do_sample=True,
        eos_token_id=tokenizer('<|im_end|>').input_ids
    )
    decoded_prediction = tokenizer.decode(prediction[0, input_ids.size(1):]).strip()
    output.clear()

    # Remove <|im_end|> from the decoded message
    decoded_prediction = decoded_prediction.replace("<|im_end|>", "")

    return decoded_prediction

In [4]:
question = "Hei, hvordan går det?" # @param {type:"string"}

output_text = generate({
    "input": question,
})

display(Markdown(output_text))

Hei! Jeg er en KI-assistent, og jeg er her for å hjelpe deg. Hvordan kan jeg bistå deg i dag? Vil du at jeg skal hjelpe deg med et spørsmål, gi informasjon, eller kanskje svare på en spørreundersøkelse? Vennligst gi meg beskjed om hvordan jeg kan hjelpe deg.

## Chat with history

In [5]:
@torch.no_grad()
def generate_with_history(input_dict):
    global conversation_history

    # Append new user input to the conversation history
    conversation_history += f"<|im_start|> user {input_dict['input']}<|im_end|>\n"

    # Prepare the full prompt by combining the conversation history and the assistant's prompt
    full_prompt = conversation_history + "<|im_start|> assistant\n"

    input_ids = tokenizer(full_prompt, return_tensors='pt').input_ids.cuda()

    # Generate the model's response
    prediction = model.generate(
        input_ids,
        max_new_tokens=2048, #adjust up or down to 4096
        temperature=0.2, # adjust up or down for creativity
        top_p=0.95, # adjust up or down for creativity
        do_sample=True,
        eos_token_id=tokenizer('<|im_end|>').input_ids
    )

    decoded_prediction = tokenizer.decode(prediction[0, input_ids.size(1):]).strip()

    # Append the assistant's answer to the conversation history
    conversation_history += f"<|im_start|> assistant {decoded_prediction}\n"

    # Remove <|im_end|> from the decoded message
    decoded_prediction = decoded_prediction.replace("<|im_end|>", "")

    # Clear the output for each new input
    output.clear()

    return decoded_prediction

In [8]:
# Run cell to clear history
conversation_history = ""

In [11]:

question_history = "Hva har vi snakket om?" # @param {type:"string"}
output_text_history = generate_with_history({
    "input": question_history,
})

display(Markdown(output_text_history))


Jeg er en KI-assistent, og jeg er her for å hjelpe deg med å svare på spørsmålene dine. Basert på informasjonen jeg har tilgang til, har vi diskutert værforholdene i området ditt. Jeg håper denne informasjonen er nyttig, og vennligst gi meg beskjed hvis du har andre spørsmål!

Merk: Denne informasjonen er basert på den nyeste værdataen som er tilgjengelig for meg på tidspunktet for dette svaret. Jeg anbefaler å sjekke en pålitelig værkilde for den mest oppdaterte og nøyaktige informasjonen.

In [9]:
print(conversation_history)

In [7]:
# Teste å kjøre en tekst til image modell.

In [2]:
!pip install diffusers

In [4]:
from diffusers import StableDiffusionPipeline
import torch
model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
prompt = "A cute puppy"
image = pipe(prompt).images[0]
image.save("generated_image.png")
print("Image saved as generated_image.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image saved as generated_image.png
